# Build pipeline with registered components

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `CommandComponent` using YAML, `command_component` decorator
- Create components into workspace
- Create `Pipeline` using registered components.

**Motivations** - This notebook explains different method to create components via SDK then use these components to build pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [27]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [28]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [29]:
# Get a handle to workspace
#ml_client = MLClient.from_config(credential=credential)

subscription_id = "0e7c68e3-ddc7-4979-bdc2-30a14addf264"
resource_group = "openai-rg"
workspace = "aml-workspace-dereknguyen"

ml_client = MLClient(credential, subscription_id, resource_group, workspace)
# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
#print(ml_client.compute.get(cluster_name))

# 2. Define and create components into workspace
## 2.1 Load components definition from YAML and Register components into workspace

#### 

In [30]:
parent_dir = "."

In [31]:
translation_component=load_component(source=parent_dir + "/2-translation-component-spec.yml")

try:
    translation_component=ml_client.components.get(name="texttranslationcomponent", version="1.1.3")
except:
    translation_component=ml_client.components.create_or_update(translation_component)


print(translation_component)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: texttranslationcomponent
version: 1.1.2
display_name: Text Translation Component
type: command
inputs:
  text:
    type: string
    optional: false
    description: The text to translate
  target_languages:
    type: string
    optional: false
    description: Target languages separated by commas
  key:
    type: string
    optional: false
    description: Azure subscription key for translation
  endpoint:
    type: string
    optional: false
    description: Azure endpoint for translation
  region:
    type: string
    optional: false
    description: Azure region for translation
outputs:
  translated_text:
    type: string
    description: Translated text
command: python 1-translation-component.py --text ${{inputs.text}} --target-languages
  ${{inputs.target_languages}} --key ${{inputs.key}} --endpoint ${{inputs.endpoint}}
  --region ${{inputs.region}} --output ${{outputs.translated_text}}
environ

##### Check if the enviroment is available and ge the enviroment id 

In [9]:
envs = ml_client.environments.get(name="docker-image-for-pair-matching", version="2")
print(envs)

conda_file:
  channels:
  - conda-forge
  dependencies:
  - python=3.8
  - pip=21.2.4
  - pip:
    - azure-ai-translation-text==1.0.0b1
    - jellyfish==0.9.0
    - joblib==1.2.0
    - numpy==1.23.4
    - pandas==1.5.1
    - python-dateutil==2.8.2
    - pytz==2022.6
    - recordlinkage==0.15
    - scikit-learn==1.1.3
    - scipy==1.9.3
    - six==1.16.0
    - threadpoolctl==3.1.0
    - matplotlib~=3.5.0
    - psutil~=5.8.0
    - tqdm~=4.62.0
    - ipykernel~=6.0
    - azureml-core==1.47.0
    - azureml-defaults==1.47.0
    - azureml-mlflow==1.47.0
    - azureml-telemetry==1.47.0
    - debugpy~=1.6.3
    - openai==0.26.4
    - pandasql==0.7.3
    - azure-ai-ml==1.1.0
    - azure-ai-formrecognizer==3.2.0
    - azure-ai-textanalytics==5.2.1
    - sqlalchemy==1.4.40
    - openpyxl==3.0.10
    - pypdf2==3.0.1
    - confluent-kafka==1.7.0
    - pymsteams==0.2.2
    - azure-search-documents==11.4.0b3
    - pypdf==3.5.0
  name: recordlinkage
creation_context:
  created_at: '2023-09-12T04:56:02

# 3. pipeline job
## 3.1 Build pipeline

##### translation pipeline

### Create the Pipeline

In [17]:
from azure.ai.ml.constants import AssetTypes

# Construct pipeline
# @pipeline()
# def pipeline_translation(
#     text: str,
#     target_languages: str,
#     key: str,
#     endpoint: str,
#     region: str
# ) -> dict:

@pipeline()
def pipeline_translation(
    text,
    target_languages,
    key,
    endpoint,
    region
):
    
    
    """TextTanslationPipeline"""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    translation = translation_component(
            text=text,
            target_languages=target_languages,
            key=key,
            endpoint=endpoint,
            region=region
     )


    # Return: pipeline outputs
    return {
        "outputs": translation.outputs.translated_text
    }

pipeline_job = pipeline_translation(
    text= "<>",
    target_languages= '<>',
    key= '<>',
    endpoint= '<>',
    region= '<>'
)

# pipeline_job = pipeline_translation(
#     text= "hello world from Microsoft!",
#     target_languages= 'es,fr',
#     key= '2dca5c799fae4bacb9d3ea41e514a0d7',
#     endpoint= 'https://api.cognitive.microsofttranslator.com/',
#     region= 'eastus'
# )

# set pipeline level compute
#pipeline_job.settings.default_compute = "ym-cluster-2"
pipeline_job.settings.default_compute = "dereknguyen1"

# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="TextTranslationPipeline"
)
pipeline_job

Exception: Value 'string' passed is not in set ['uri_file', 'uri_folder', 'custom_model', 'mlflow_model', 'mltable', 'triton_model']

In [26]:
@pipeline()
def pipeline_translation(
    text: str,
    target_languages: str,
    key: str,
    endpoint: str,
    region: str
) -> Output(type="uri_file"):
    """TextTanslationPipeline"""
    
    translation = translation_component(
        text=text,
        target_languages=target_languages,
        key=key,
        endpoint=endpoint,
        region=region
    )

    return {
        "output": translation.outputs.translated_text
    }

# Replace with actual values
text_input = "Hello World!"
target_languages_input = "es,fr"
key_input = "your-azure-key"
endpoint_input = "https://api.cognitive.microsofttranslator.com/"
region_input = "eastus"

pipeline_job = pipeline_translation(
    text=text_input,
    target_languages=target_languages_input,
    key=key_input,
    endpoint=endpoint_input,
    region=region_input
)

pipeline_job.settings.default_compute = "dereknguyen1"

# Assuming ml_client and experiment_name are already defined
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="TextTranslationPipeline"
)

print(pipeline_job)

Exception: Value 'string' passed is not in set ['uri_file', 'uri_folder', 'custom_model', 'mlflow_model', 'mltable', 'triton_model']

# Next Steps
You can see further examples of running a pipeline job [here](../)